# Jpeg algorithm

**Groupe 30**: Tony Heng / Gabriel Rayzal / Nathan Cabasso / Ferdinand Mom

In [1]:
# Explained what is JPEG etc ...

# Table of Contents

- I/ Compression
- II/ Decompression
- III/ Test on real image

# <ins> I/ Compression </ins>

Whole compression pipeline

![](https://cdn.discordapp.com/attachments/844292513665515531/859104978538856448/unknown.png)

## 1) Block splitting

First step is to divide input image into non-overlapping 8 × 8 macro-blocks.
![](https://cdn.discordapp.com/attachments/844292513665515531/859064441525370900/unknown.png)
If the dimensions are not divisible in integer numbers of blocks, the image can be padded:

- zero padding: #TODO
- replicating border: #TODO

A naive approach could be to pad only a side of the image (bottom-left). The downside of this approach is that we can find ourselves with macro-blocks full of zeros. Thus, we decided to implement our padding in a such way that the image is always center in the middle.

## 2) DCT

The second step is to perform a DCT (Discret Cosine Transformation) on each macro-block. To understand the need of DCT, we need to first understand what is an image. An image is the superposition of:

- Low frequencies: overall structure of image.
- High frequencies: Details (edges and noises)

An image is originally represented in its spatial representation which means the image is decomposed in a set of basis image.

![](https://cdn.discordapp.com/attachments/844292513665515531/859090327977721866/unknown.png)

The problem is then the following: How do we discard the useless part of high frequency (noises) without removing important information ?

To do so, we perform a frequency analysis of the image. However, the way the image is initially represented (in spatial representation) is not suited to carry out a frequency analysis. Indeed, high frequencies are disseminated everywhere in the image so it is hard to tell whether a quick variation in terms of pixel is due to an edge (important) or noise (to be discarded).

That is why we need to change the representation of our image which is done by performing transformations on our image in order to represent it into a new basis. More precisely, the transformation will enable to represent our image into a new basis. Here are some transformations:

- Walsh-Hadamard
- Discret Fourier Transform (DFT)
- Discret Cosine Transform (DCT)

JPEG uses DCT which succeed in compacting most of the image energy into as few coefficients as possible compare to Waslh-Hadamard and does not require to store complex number (real and imaginary parts) like DFT which requires double the memory for storage.

Each 8x8 macro-block will then be multiply with the following DCT kernel matrix:

![](https://cdn.discordapp.com/attachments/844292513665515531/859082211910221824/unknown.png)

The result of the DCT is also an 8 by 8 block where the first coefficient at the top left corner represents the DC coefficient. Remaining coefficients are called AC coefficients. The advantage of the DCT is its tendency to aggregate most of the signal in top left corner of the result. Note that the higher the coefficient, the more information it contains, useful to properly reconstruct the image. Thus, we can gain in storage by removing smaller coefficients which is the goal of next step.

## 3) Quantization

The third step is quantization which looks for a way to maximize the number of 0 coefficients. This is done through the use of quantization matrix which were found empirically.

First quantization matrix is for RGB image while second one is for YUV image (only to be applied to UV channels): 

![](https://cdn.discordapp.com/attachments/844292513665515531/859100308390805534/quantification_Qq.png)
![](https://cdn.discordapp.com/attachments/844292513665515531/859100870369214534/quantification_Quv.png)

The formula is the following:

$$round(\frac{DCT(block - 128)}{Q})$$

- Before applying DCT, we must remove 128 to each 8x8 macro-block because statiscally, it was found that the mean value of an 8x8 macro-block is close to 128. Thus, $DCT(block - 128)$ wille result in more value close to 0.
- After dividing by the quantization matrix, we round to the nearest integer. This is the reason why JPEG is a lossy algorithm. It will not be possible to recover the same value during decompression.

One can also change the quality of compression using a quality factor $q \in [1, 100]$ and the following algorithm:

![](https://cdn.discordapp.com/attachments/844292513665515531/859104454381142046/unknown.png)

The higher $q$ is, the more similar the compressed image will be compared to the original image.

## 4) Zigzag + Huffman

### a) Zigzag

The goal of this step is to represent our 8x8 macro-block with only the essential coefficients. Indeed, after quantization, a lot of coefficients on the bottom corner of our macro-block are set to 0 while most of the important coefficients are on the top left corner.

![](https://cdn.discordapp.com/attachments/844292513665515531/859105773091291146/unknown.png)

### b) Huffman

We know need to encode coefficients in the zigzag ordering using JPEG predefined Huffman table for DC/AC coefficients. We used tables from "Digital Image Processing" by Rafael C. Gonzalez & Richard E. Woods page 934.

Let's denote:

- **A**: JPEG coefficients coding categories table
- **B**: JPEG DC coefficients table
- **C**: JPEG AC coefficients table

**<ins>To encode DC coefficient<ins>:** $DC = -26$
- Check in **A** in which category <a style="color:green">CAT</a> $-26$ belongs to: (category 5).
    - This means that -26 needs 5 bits to be represented in binary.
    - Retrieve <a style="color:red">binary representation</a> of number with <a style="color:green">CAT</a> in **A** (<a style="color:red">00101</a>)
    ![](https://cdn.discordapp.com/attachments/844292513665515531/859111745871413278/unknown.png)
- Retrieve codeword in **B** using <a style="color:green">CAT</a> $\to$ codeword = <mark> <a style="color:green">110</a></mark>
    - Conclusion: $DC =$ <mark><a style="color:green">110</a></mark> <a style="color:red">00101</a>
    
**<ins>To encode AC coefficients<ins>:** $AC = -3$ (The second $-3$ on above zigzag ordering)
- Check in **A** in which category <a style="color:green">CAT</a> $-3$ belongs to: (category 2)
  - This means that -3 needs 2 bits to be represented in binary.
    - Retrieve <a style="color:red">binary representation</a> of number with <a style="color:green">CAT</a> in **B** (<a style="color:red">00</a>)
    ![](https://cdn.discordapp.com/attachments/844292513665515531/859113073365614602/unknown.png)
- Compute <a style="color:purple">RUN</a> which correspond to number of 0s precedding AC coeff (<a style="color:purple">RUN=1</a>)
- Retrieve codeword in **C** using <mark><a style="color:purple">RUN</a>/<a style="color:green">CAT</a> = <a style="color:purple">1</a>/<a style="color:green">2</a></mark> $\to$ codeword = <mark>111001</mark>
- Conclusion: $AC = $ <mark>111001</mark><a style="color:red">00</a>

# <ins> II/ Decompresion </ins>

![](https://cdn.discordapp.com/attachments/844292513665515531/859105375265226782/unknown.png)

## 1) Huffman$^{-1}$ + Zigzag$^{-1}$

### A) Huffman$^{-1}$

### b) Zigzag$^{-1}$

## 2) Reverse quantization

## 3) DCT$^{-1}$

## 4) Block combination